In [1]:
import cv2
import numpy as np
import mediapipe as mp
import os
from typing import Tuple, List, Dict, Optional

# 初始化mediapipe人脸网格模型
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True)

def get_face_landmarks(image: np.ndarray) -> Optional[Dict[int, Tuple[int, int]]]:
    """获取人脸的所有关键点坐标"""
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if not results.multi_face_landmarks:
        return None
    
    landmarks = results.multi_face_landmarks[0].landmark
    h, w = image.shape[:2]
    
    return {i: (int(landmark.x * w), int(landmark.y * h)) for i, landmark in enumerate(landmarks)}

def apply_hair_overlay(
    face_img: np.ndarray,
    hair_img: np.ndarray,
    hair_anchor_points: List[Tuple[int, int]],
    face_landmarks: Dict[int, Tuple[int, int]]
) -> np.ndarray:
    """
    将发型图片应用到人脸图片上
    :param face_img: 原始人脸图片 (BGR格式)
    :param hair_img: 发型图片 (PNG格式，带alpha通道)
    :param hair_anchor_points: 发型图片上的两个锚点 [(x1,y1), (x2,y2)]
    :param face_landmarks: 人脸关键点字典
    :return: 处理后的图片
    """
    # 确保发型图片有alpha通道
    if hair_img.shape[2] == 3:
        hair_img = cv2.cvtColor(hair_img, cv2.COLOR_BGR2BGRA)
    
    # 获取人脸关键点234和454
    point_234 = face_landmarks.get(234)
    point_454 = face_landmarks.get(454)
    if not point_234 or not point_454:
        raise ValueError("缺少必要的人脸关键点(234或454)")
    
    face_anchor_points = [point_234, point_454]
    
    # 计算变换矩阵
    src_vec = np.array(hair_anchor_points[1]) - np.array(hair_anchor_points[0])
    dst_vec = np.array(face_anchor_points[1]) - np.array(face_anchor_points[0])
    
    src_length = np.linalg.norm(src_vec)
    dst_length = np.linalg.norm(dst_vec)
    scale = dst_length / src_length
    
    angle = np.arctan2(dst_vec[1], dst_vec[0]) - np.arctan2(src_vec[1], src_vec[0])
    angle = -np.degrees(angle)
    
    center = ((hair_anchor_points[0][0] + hair_anchor_points[1][0]) / 2, 
              (hair_anchor_points[0][1] + hair_anchor_points[1][1]) / 2)
    
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, scale)
    
    tx = face_anchor_points[0][0] - (rotation_matrix[0][0] * hair_anchor_points[0][0] + 
                                    rotation_matrix[0][1] * hair_anchor_points[0][1] + 
                                    rotation_matrix[0][2])
    ty = face_anchor_points[0][1] - (rotation_matrix[1][0] * hair_anchor_points[0][0] + 
                                    rotation_matrix[1][1] * hair_anchor_points[0][1] + 
                                    rotation_matrix[1][2])
    
    rotation_matrix[0][2] += tx
    rotation_matrix[1][2] += ty
    
    # 获取人脸图片尺寸
    h, w = face_img.shape[:2]
    
    # 变换发型图片
    transformed_hair = cv2.warpAffine(hair_img, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR)
    
    # 分离alpha通道
    hair_rgb = transformed_hair[:, :, :3]
    hair_alpha = transformed_hair[:, :, 3] / 255.0
    
    # 确保人脸图片有alpha通道
    if face_img.shape[2] == 3:
        face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2BGRA)
    
    # 创建结果图像
    result = face_img.copy()
    
    # 使用alpha混合
    for c in range(0, 3):
        result[:, :, c] = (1 - hair_alpha) * face_img[:, :, c] + hair_alpha * hair_rgb[:, :, c]
    
    # 裁剪超出边界的部分
    result = result[:h, :w]
    
    return cv2.cvtColor(result, cv2.COLOR_BGRA2BGR)

def process_all_images_with_hair(
    main_dir: str,
    output_base_dir: str,
    hair_path: str
):
    """
    批量处理主目录下所有子文件夹中的图片，添加发型
    :param main_dir: 主目录路径
    :param output_base_dir: 输出基础目录
    :param hair_path: 发型模板图片路径
    """
    try:
        hair_img = cv2.imread(hair_path, cv2.IMREAD_UNCHANGED)
        if hair_img is None:
            raise ValueError(f"无法读取发型图片: {hair_path}")
    except Exception as e:
        print(f"加载发型图片错误: {str(e)}")
        return
    
    # 发型图片锚点
    hair_anchor_points = [(255, 1531), (1601, 1531)]
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    
    # 获取主目录下所有包含'2'的子文件夹
    subdirs = [d for d in os.listdir(main_dir) 
              if os.path.isdir(os.path.join(main_dir, d)) and '2' in d]
    
    if not subdirs:
        print(f"主目录中没有找到符合条件的子文件夹: {main_dir}")
        return
    
    total_processed = 0
    total_images = 0
    
    for subdir in subdirs:
        input_dir = os.path.join(main_dir, subdir)
        output_dir = os.path.join(output_base_dir, subdir)
        os.makedirs(output_dir, exist_ok=True)
        
        # 获取子文件夹中所有支持的图片文件
        image_files = [f for f in os.listdir(input_dir) 
                      if f.lower().endswith(supported_extensions)]
        
        if not image_files:
            print(f"子文件夹中没有图片文件: {input_dir}")
            continue
        
        processed_count = 0
        print(f"\n正在处理文件夹: {subdir}")
        
        for filename in image_files:
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
            
            try:
                face_img = cv2.imread(input_path)
                if face_img is None:
                    print(f"  无法读取图片: {filename}")
                    continue
                
                face_landmarks = get_face_landmarks(face_img)
                if not face_landmarks:
                    print(f"  未检测到人脸: {filename}")
                    continue
                
                result = apply_hair_overlay(face_img, hair_img, hair_anchor_points, face_landmarks)
                
                if not cv2.imwrite(output_path, result):
                    print(f"  无法保存结果: {filename}")
                    continue
                
                processed_count += 1
                print(f"  已处理: {filename}")
                
            except Exception as e:
                print(f"  处理图片 {filename} 时出错: {str(e)}")
        
        total_processed += processed_count
        total_images += len(image_files)
        print(f"文件夹 {subdir} 处理完成: {processed_count}/{len(image_files)}")
    
    print(f"\n全部处理完成! 总计处理 {total_processed}/{total_images} 张图片")

if __name__ == "__main__":
    # 配置参数
    main_directory = "./"  # 主文件夹路径
    output_directory = "futures_blur"  # 输出文件夹路径
    hair_image_path = "hair.png"  # 发型模板图片路径
    
    print("开始批量处理所有子文件夹中的图片(发型处理)...")
    process_all_images_with_hair(main_directory, output_directory, hair_image_path)

开始批量处理所有子文件夹中的图片(发型处理)...

正在处理文件夹: 2019
  已处理: 01.jpg
文件夹 2019 处理完成: 1/1

全部处理完成! 总计处理 1/1 张图片
